In [3]:
%pip install accelerate

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 7.4 MB/s eta 0:00:00a 0:00:01
DEPRECATION: flatbuffers 1.12.1-git20200711.33e2d80-dfsg1-0.6 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of flatbuffers or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

In [2]:
model_name = "deepseek-ai/deepseek-math-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto")
model.generation_config = GenerationConfig.from_pretrained(model_name)
model.generation_config.pad_token_id = model.generation_config.eos_token_id

pytorch_model.bin.index.json:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

/home/ubuntu/.local/lib/python3.10/site-packages/accelerate/utils/modeling.py:1462: UserWarning: Current model requires 3968 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/lib/python3/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

In [3]:
prompt_1 = """Here is a question, presented with informal langage and formal Lean 4 code.

/-!
# International Mathematical Olympiad 1963, Problem 5

Prove that cos(π/7) - cos(2π/7) + cos(3π/7) = 1/2.
-/
```lean4
namespace Imo1963P5

open scoped Real

problem imo1963_p5 :
    Real.cos (π/7) - Real.cos (2*π/7) + Real.cos (3*π/7) = 1/2 := by
```

A student is attempting to solve this problem, and has gotten this far:

```lean4
  rw [show (2*π/7) = π - (5*π/7) by linarith]
  rw [Real.cos_pi_sub]
  simp only [sub_neg_eq_add]
  have h : 2 * Real.sin (π / 7) ≠ 0 := by
    simp only [ne_eq, mul_eq_zero, OfNat.ofNat_ne_zero, false_or]
    apply ne_of_gt
    apply Real.sin_pos_of_pos_of_lt_pi
    simp only [Nat.ofNat_pos, div_pos_iff_of_pos_right, Real.pi_pos]
    trans 1
    · rw [div_lt_one (by linarith only)]
      linarith only [Real.pi_le_four]
    · linarith only [Real.pi_gt_three]
  apply (mul_right_inj' h).mp
  rw [left_distrib, left_distrib]
  have prod_sum : ∀ (x y : ℝ),
      2 * Real.sin x * Real.cos y = Real.sin (x + y) - Real.sin (y - x) := by
    intro x y
    rw [Real.sin_add, Real.sin_sub]
    linarith only
  rw [prod_sum, prod_sum, prod_sum]
```
Please summarize the progress the student has made up to this point.
"""

prompt_2 = """
Please suggest 5 distinct proof ideas which can finish from this point. Be sure to delimit your ideas with <IDEA> </IDEA>.
"""

prompt = prompt_1 + prompt_2

In [6]:
messages = [
    {"role": "user", "content": prompt}
]
input_tensor = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
outputs = model.generate(input_tensor.to(model.device), max_new_tokens=100)

result = tokenizer.decode(outputs[0][input_tensor.shape[1]:], skip_special_tokens=True)
print(result)

KeyboardInterrupt: 

In [3]:
messages = [
    {"role": "user", "content": "what is the integral of x^2 from 0 to 2?\nPlease reason step by step, and put your final answer within \\boxed{}."}
]
input_tensor = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
outputs = model.generate(input_tensor.to(model.device), max_new_tokens=100)

result = tokenizer.decode(outputs[0][input_tensor.shape[1]:], skip_special_tokens=True)
print(result)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


KeyboardInterrupt: 

Why are these not terminating?

Base

In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

model_name = "deepseek-ai/deepseek-math-7b-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto")
model.generation_config = GenerationConfig.from_pretrained(model_name)
model.generation_config.pad_token_id = model.generation_config.eos_token_id

text = "The integral of x^2 from 0 to 2 is"
inputs = tokenizer(text, return_tensors="pt")
outputs = model.generate(**inputs.to(model.device), max_new_tokens=100)

result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(result)

tokenizer_config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

/home/ubuntu/.local/lib/python3.10/site-packages/accelerate/utils/modeling.py:1462: UserWarning: Current model requires 3968 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

KeyboardInterrupt: 

In [9]:
pip install vllm

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.3/228.3 MB 18.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 5.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 109.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 90.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 11.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 101.5 MB/s eta 0

In [13]:
%pip install typing_extensions --upgrade

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: flatbuffers 1.12.1-git20200711.33e2d80-dfsg1-0.6 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of flatbuffers or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from vllm import LLM, SamplingParams

llm = LLM(model="deepseek-ai/DeepSeek-Prover-V1.5-RL",
              max_num_batched_tokens=8192,
              trust_remote_code=True)

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


INFO 09-29 04:57:21 llm_engine.py:226] Initializing an LLM engine (v0.6.1.dev238+ge2c6e0a82) with config: model='deepseek-ai/DeepSeek-Prover-V1.5-RL', speculative_config=None, tokenizer='deepseek-ai/DeepSeek-Prover-V1.5-RL', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=deepseek-ai/DeepSeek-Prover-V1.5-RL, use_v2_block_manager=False, num_scheduler_steps=

model-00002-of-000002.safetensors:   0%|          | 0.00/5.23G [00:00<?, ?B/s]

model-00001-of-000002.safetensors:   0%|          | 0.00/8.59G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/22.7k [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 09-29 05:00:24 model_runner.py:1025] Loading model weights took 12.8725 GB
INFO 09-29 05:00:25 gpu_executor.py:122] # GPU blocks: 2895, # CPU blocks: 546
INFO 09-29 05:00:31 model_runner.py:1329] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 09-29 05:00:31 model_runner.py:1333] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 09-29 05:00:48 model_runner.py:1456] Graph capturing finished in 17 secs.


In [32]:
sampling_params=SamplingParams(
    max_tokens=4000,
    temperature=0.0,
    top_k=1,
    top_p=1.0
)

In [4]:
outputs = llm.generate(
    prompt,
    sampling_params=sampling_params
)

Processed prompts: 100%|██████████| 1/1 [00:05<00:00,  5.10s/it, est. speed input: 108.51 toks/s, output: 78.49 toks/s]


In [8]:
outputs[0].__dict__.keys()

dict_keys(['request_id', 'prompt', 'prompt_token_ids', 'prompt_logprobs', 'outputs', 'finished', 'metrics', 'lora_request', 'encoder_prompt', 'encoder_prompt_token_ids'])

In [12]:
print(outputs[0].outputs[0].text)

The student has made progress by rewriting the expression using trigonometric identities and simplifying the resulting expression. The key steps involve:

1. Rewriting \(2\pi/7\) as \(\pi - 5\pi/7\).
2. Using the identity \(\cos(\pi - x) = -\cos(x)\) to simplify the expression.
3. Applying the product-to-sum identities to further simplify the expression.

Here are five distinct proof ideas to finish the proof:

<IDEA 1>
Use the product-to-sum identities to express the sum of cosines as a single sine function.

<IDEA 2>
Simplify the resulting expression using known values of trigonometric functions.

<IDEA 3>
Use algebraic manipulation to show that the simplified expression equals \(1/2\).

<IDEA 4>
Verify the final result using known trigonometric identities and properties.

<IDEA 5>
Conclude the proof by showing that the simplified expression matches the desired result.
Let's go through each idea in detail:

<IDEA 1>
Use the product-to-sum identities to express the sum of cosines as a

In [33]:
prompt = """ Here is a question, presented with informal langage and formal Lean 4 code.

/-!
# International Mathematical Olympiad 1963, Problem 5

Prove that cos(π/7) - cos(2π/7) + cos(3π/7) = 1/2.
-/

```lean4
namespace Imo1963P5

open scoped Real

problem imo1963_p5 :
    Real.cos (π/7) - Real.cos (2*π/7) + Real.cos (3*π/7) = 1/2 := by
```

A smart student is attempting to solve this problem, and has gotten this far:

```lean4
  rw [show (2*π/7) = π - (5*π/7) by linarith]
  rw [Real.cos_pi_sub]
  simp only [sub_neg_eq_add]
  have h : 2 * Real.sin (π / 7) ≠ 0 := by
    simp only [ne_eq, mul_eq_zero, OfNat.ofNat_ne_zero, false_or]
    apply ne_of_gt
    apply Real.sin_pos_of_pos_of_lt_pi
    simp only [Nat.ofNat_pos, div_pos_iff_of_pos_right, Real.pi_pos]
    trans 1
    · rw [div_lt_one (by linarith only)]
      linarith only [Real.pi_le_four]
    · linarith only [Real.pi_gt_three]
  apply (mul_right_inj' h).mp
  rw [left_distrib, left_distrib]
  have prod_sum : ∀ (x y : ℝ),
      2 * Real.sin x * Real.cos y = Real.sin (x + y) - Real.sin (y - x) := by
    intro x y
    rw [Real.sin_add, Real.sin_sub]
    linarith only
  rw [prod_sum, prod_sum, prod_sum]
```

At this point, the student is considering the following possible ways to finish the proof:

The student has made progress by rewriting the expression using trigonometric identities and simplifying the resulting expression. The key steps involve:

1. Rewriting \(2\pi/7\) as \(\pi - 5\pi/7\).
2. Using the identity \(\cos(\pi - x) = -\cos(x)\) to simplify the expression.
3. Applying the product-to-sum identities to further simplify the expression.

Here are five distinct proof ideas to finish the proof:

<IDEA 1>
Use the product-to-sum identities to express the sum of cosines as a single sine function.

<IDEA 2>
Simplify the resulting expression using known values of trigonometric functions.

<IDEA 3>
Use algebraic manipulation to show that the simplified expression equals \(1/2\).

<IDEA 4>
Verify the final result using known trigonometric identities and properties.

<IDEA 5>
Conclude the proof by showing that the simplified expression matches the desired result.

Here is the correct completion to this proof:
```lean4
  rw [show (π / 7 + π / 7)     = 2 * π / 7 by linarith only]
  rw [show (π / 7 - π / 7)     = 0         by linarith only]
  rw [show (π / 7 + 5 * π / 7) = 6 * π / 7 by linarith only]
  rw [show (5 * π / 7 - π / 7) = 4 * π / 7 by linarith only]
  rw [show (π / 7 + 3 * π / 7) = 4 * π / 7 by linarith only]
  rw [show (3 * π / 7 - π / 7) = 2 * π / 7 by linarith only]
  rw [Real.sin_zero]
  ring_nf
  rw [← Real.sin_pi_sub]
  rw [show (π - π * (6 / 7)) = π / 7 by linarith]
  congr
  linarith
end Imo1963P5
```

How well does each idea capture the remaining steps of the lean proof? Output 5 scores which should be a number from 0 to 100. For example,
<CONFIDENCE>
1. 11
2. 32
3. 61
4. 05
5. 99
</CONFIDENCE>
"""
outputs = llm.generate(
    prompt,
    sampling_params=sampling_params
)
print(outputs[0].outputs[0].text)

Processed prompts: 100%|██████████| 1/1 [00:07<00:00,  7.32s/it, est. speed input: 151.58 toks/s, output: 77.43 toks/s]

### Analysis

The given Lean4 code and the informal proof outline provide a good starting point for solving the problem. The proof strategy involves rewriting the expression using trigonometric identities and simplifying the resulting expression. The key steps include:

1. Rewriting \(2\pi/7\) as \(\pi - 5\pi/7\).
2. Using the identity \(\cos(\pi - x) = -\cos(x)\) to simplify the expression.
3. Applying the product-to-sum identities to further simplify the expression.

The proof outline suggests using the product-to-sum identities to express the sum of cosines as a single sine function. This approach is valid and aligns with standard trigonometric techniques.

### Evaluation of Proof Ideas

1. **Idea 1**: Using the product-to-sum identities to express the sum of cosines as a single sine function.
   - This idea is sound and aligns with standard trigonometric techniques.

2. **Idea 2**: Simplifying the resulting expression using known values of trigonometric functions.
   - This step is

pretty good

In [25]:
prompt = """
Explain what this is doing in one sentence.
```lean4
  rw [show (π / 7 + π / 7)     = 2 * π / 7 by linarith only]
  rw [show (π / 7 - π / 7)     = 0         by linarith only]
  rw [show (π / 7 + 5 * π / 7) = 6 * π / 7 by linarith only]
  rw [show (5 * π / 7 - π / 7) = 4 * π / 7 by linarith only]
  rw [show (π / 7 + 3 * π / 7) = 4 * π / 7 by linarith only]
  rw [show (3 * π / 7 - π / 7) = 2 * π / 7 by linarith only]
  rw [Real.sin_zero]
  ring_nf
  rw [← Real.sin_pi_sub]
  rw [show (π - π * (6 / 7)) = π / 7 by linarith]
  congr
  linarith
"""

In [26]:
outputs = llm.generate(
    prompt,
    sampling_params=sampling_params
)
print(outputs[0].outputs[0].text)

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.92s/it, est. speed input: 67.91 toks/s, output: 79.40 toks/s]

```The given Lean4 code is a sequence of rewrites and simplifications applied to a trigonometric expression. The purpose is to simplify the expression involving sine functions and constants. The steps involve breaking down the expression into simpler components, applying known trigonometric identities, and simplifying the resulting expression.

### Explanation of Each Step

1. **First Rewrite**: Combines two terms using basic arithmetic to simplify the expression.
2. **Second Rewrite**: Shows that a subtraction of the same term results in zero.
3. **Third Rewrite**: Combines two terms to form a simpler expression.
4. **Fourth Rewrite**: Shows that a subtraction of the same term results in zero.
5. **Fifth Rewrite**: Combines two terms to form a simpler expression.
6. **Sixth Rewrite**: Shows that a subtraction of the same term results in zero.
7. **Sine of Zero**: Uses the fact that the sine of zero is zero.
8. **Normalization**: Simplifies the expression using ring normalization.
9. *

In [28]:
prompt = """
Explain what this is doing to someone who doesn't know lean:
```lean4
  rw [show (π / 7 + π / 7)     = 2 * π / 7 by linarith only]
"""
outputs = llm.generate(
    prompt,
    sampling_params=sampling_params
)
print(outputs[0].outputs[0].text)

Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.98s/it, est. speed input: 10.43 toks/s, output: 80.27 toks/s]

  rw [show (2 * π / 7 + 2 * π / 7) = 4 * π / 7 by linarith only]
  rw [show (4 * π / 7 + 4 * π / 7) = 8 * π / 7 by linarith only]
  rw [show (8 * π / 7 + 8 * π / 7) = 16 * π / 7 by linarith only]
  rw [show (16 * π / 7 + 16 * π / 7) = 32 * π / 7 by linarith only]
  rw [show (32 * π / 7 + 32 * π / 7) = 64 * π / 7 by linarith only]
  rw [show (64 * π / 7 + 64 * π / 7) = 128 * π / 7 by linarith only]
  rw [show (128 * π / 7 + 128 * π / 7) = 256 * π / 7 by linarith only]
  rw [show (256 * π / 7 + 256 * π / 7) = 512 * π / 7 by linarith only]
  rw [show (512 * π / 7 + 512 * π / 7) = 1024 * π / 7 by linarith only]
  rw [show (1024 * π / 7 + 1024 * π / 7) = 2048 * π / 7 by linarith only]
  rw [show (2048 * π


hm